Allison Forte

January 26, 2023

DSC 550

Exercise 7.2 

In [226]:
import pandas as pd
import numpy as np
import category_encoders as cat_encoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

# Part 1: PCA and Variance Threshold in a Linear Regression

1. Import the housing data as a data frame and ensure that the data is loaded properly.
2. Drop the "Id" column and any features that are missing more than 40% of their values.
3. For numerical columns, fill in any missing data with the median value.
4. For categorical columns, fill in any missing data with the most common value (mode).
5. Convert the categorical columns to dummy variables.
6. Split the data into a training and test set, where the SalePrice column is the target.
7. Run a linear regression and report the R2-value and RMSE on the test set.
8. Fit and transform the training features with a PCA so that 90% of the variance is retained (see section 9.1 in the Machine Learning with Python Cookbook).
9. How many features are in the PCA-transformed matrix?
10. Transform but DO NOT fit the test features with the same PCA.
11. Repeat step 7 with your PCA transformed data.
12. Take your original training features (from step 6) and apply a min-max scaler to them.
13. Find the min-max scaled features in your training set that have a variance above 0.1 (see Section 10.1 in the Machine Learning with Python Cookbook).
14. Transform but DO NOT fit the test features with the same steps applied in steps 11 and 12.
15. Repeat step 7 with the high variance data.
16. Summarize your findings.

In [227]:
# Import the housing data as a data frame and ensure that the data is loaded properly.

housing_data = pd.read_csv('/Users/allison.forte/Downloads/house-prices-advanced-regression-techniques/train.csv')
print(housing_data.shape)
housing_data.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [229]:
# Drop the "Id" column and any features that are missing more than 40% of their values.

housing_data = housing_data.drop('Id', axis = 1)

new_data = housing_data.dropna(axis = 'columns', thresh = housing_data.shape[0]*.4)

new_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [230]:
# For numerical columns, fill in any missing data with the median value.

new_data = new_data.fillna(new_data.median())

/var/folders/nj/3lbsz0_s67186v8nxnkcdy48p4fdg7/T/ipykernel_3837/580268706.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  new_data = new_data.fillna(new_data.median())


In [231]:
# For categorical columns, fill in any missing data with the most common value (mode).

new_data = new_data.fillna(new_data.mode().iloc[0])

In [232]:
# Convert the categorical columns to dummy variables.

new_data = pd.get_dummies(new_data)

print(new_data.shape)
new_data.head()

(1460, 276)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


In [235]:
# Split the data into a training and test set, where the SalePrice column is the target.

y = new_data[['SalePrice']]
x = new_data.drop('SalePrice', axis = 1)  

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size = 0.2)

In [236]:
# Run a linear regression and report the R2-value and RMSE on the test set.

model = LinearRegression()
model.fit(x_train, y_train)

print('R squared for the test data is', model.score(x_test, y_test)) 
print('RMSE for the test data is', mean_squared_error(y_test, model.predict(x_test), squared=False)) 

R squared for the test data is 0.873758065741896
RMSE for the test data is 28110.77130058296


In [237]:
# Fit and transform the training features with a PCA so that 90% of the variance is retained 
# (see section 9.1 in the Machine Learning with Python Cookbook).

x_train_scaled = StandardScaler().fit_transform(x_train)

pca = PCA(n_components=.9)

principalComponents_train = pca.fit_transform(x_train_scaled)

In [238]:
# How many features are in the PCA-transformed matrix?

print('Original number of features:', x_train.shape[1])
print('Reduced number of features:', principalComponents_train.shape[1])

Original number of features: 275
Reduced number of features: 141


In [239]:
# Transform but DO NOT fit the test features with the same PCA.

x_test_scaled = StandardScaler().fit_transform(x_test)

principalComponents_test = pca.transform(x_test_scaled)

In [240]:
# Repeat step 7 with your PCA transformed data.
# (Run a linear regression and report the R2-value and RMSE on the test set.)

PCA_model = LinearRegression()
PCA_model.fit(principalComponents_train, y_train)

print('R squared for the test data with the PCA model is', PCA_model.score(principalComponents_test, y_test)) 
print('RMSE for the test data with the PCA model is', mean_squared_error
      (y_test, PCA_model.predict(principalComponents_test), squared=False)) 

R squared for the test data with the PCA model is 0.8460804884540689
RMSE for the test data with the PCA model is 31039.718937136302


In [241]:
# Take your original training features (from step 6) and apply a min-max scaler to them.

scaler = MinMaxScaler()

x_train_minmax = scaler.fit_transform(x_train)

In [242]:
# Find the min-max scaled features in your training set that have a variance above 0.1 
# (see Section 10.1 in the Machine Learning with Python Cookbook).

thresholder = VarianceThreshold(threshold = .1)

x_train_high_variance = thresholder.fit_transform(x_train_minmax)

print('Original number of features:', x_train.shape[1])
print('Reduced number of features:', x_train_high_variance.shape[1])

Original number of features: 275
Reduced number of features: 51


In [243]:
# Transform but DO NOT fit the test features with the same steps applied in steps 12 (Take your original 
# training features (from step 6) and apply a min-max scaler to them) and 13 (Find the min-max scaled features 
# in your training set that have a variance above 0.1)

x_test_minmax = scaler.fit_transform(x_test)

x_test_high_variance = thresholder.transform(x_test_minmax)

In [244]:
# Repeat step 7 with the high variance data.
# (Run a linear regression and report the R2-value and RMSE on the test set.)

high_variance_model = LinearRegression()
high_variance_model.fit(x_train_high_variance, y_train)

print('R squared for the test data with the high variance model is', high_variance_model.score
      (x_test_high_variance, y_test)) 
print('RMSE for the test data with the high variance model is', mean_squared_error
      (y_test, high_variance_model.predict(x_test_high_variance), squared=False)) 

R squared for the test data with the high variance model is 0.6318828419213149
RMSE for the test data with the high variance model is 48002.51280273474


### Summarize your findings.

Based on the R squared scores and the RMSE values for each model we can see that the original model and the PCA model both account for a high percent of the variance in the data. The final model with the high variance data accounts for less of the overall variance. 
Using fewer variables, it makes sense that less of the variability in the target can be accounted for. 
However, the model with fewer features (the high variance model) still predicts the results better than simply predicting the mean every time. 
Based on these results it would seem that the PCA model would be the best choice of the 3 models created using this data. 

# Part 2: Categorical Feature Selection

Download the data from this link Mushroom Classification. Based on several categorical features, you will predict whether or not a mushroom is edible or poisonous.
Import the data as a data frame and ensure it is loaded correctly.
Convert the categorical features (all of them) to dummy variables.
Split the data into a training and test set.
Fit a decision tree classifier on the training set.
Report the accuracy and create a confusion matrix for the model prediction on the test set.
Create a visualization of the decision tree.
Use a χ2-statistic selector to pick the five best features for this data (see section 10.4 of the Machine Learning with Python Cookbook).
Which five features were selected in step 7? Hint: Use the get_support function.
Repeat steps 4 and 5 with the five best features selected in step 7.
Summarize your findings.

In [284]:
# Import the data as a data frame and ensure it is loaded correctly.

mushroom_data = pd.read_csv('/Users/allison.forte/Downloads/mushrooms.csv')

print(mushroom_data.shape)
mushroom_data.head()

(8124, 23)


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [285]:
# Convert the categorical features (all of them) to dummy variables.

mushroom_data_dummies = pd.get_dummies(mushroom_data)

print(mushroom_data_dummies.shape)
mushroom_data_dummies.head()

(8124, 119)


,class_e,class_p,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [295]:
#  Split the data into a training and test set.
mushroom_data_dummies['label'] = mushroom_data_dummies.class_e
mushroom_data_dummies = mushroom_data_dummies.drop(['class_e', 'class_p'], axis = 1)
mushroom_data_dummies.head()

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w,label
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
3,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1


In [298]:
mush_y = mushroom_data_dummies[['label']]
mush_x = mushroom_data_dummies.drop('label', axis = 1)  

mush_x_train, mush_x_test, mush_y_train, mush_y_test = train_test_split(mush_x, mush_y, test_size = 0.2)

In [306]:
# Fit a decision tree classifier on the training set.

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

classifier = DecisionTreeClassifier()

classModel = classifier.fit(mush_x_train, mush_y_train)

y_pred = classModel.predict(mush_x_test)

In [307]:
# Report the accuracy and create a confusion matrix for the model prediction on the test set.

print("Accuracy:",metrics.accuracy_score(mush_y_test, y_pred))

print(confusion_matrix(mush_y_test, y_pred))

Accuracy: 1.0
[[758   0]
 [  0 867]]


In [335]:
# Create a visualization of the decision tree.

dot_data = tree.export_graphviz(classifier, out_file = None,  
                                feature_names = mush_x.columns, 
                                class_names = mushroom_data_dummies.columns)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

Password:
sudo: a password is required


InvocationException: GraphViz's executables not found

In [336]:
#  Use a χ2-statistic selector to pick the five best features for this data 
# (see section 10.4 of the Machine Learning with Python Cookbook).

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

features = mush_x.astype(int)
chi2_selector = SelectKBest(chi2, k=5)
features_kBest = chi2_selector.fit_transform(features, mush_y)

print('Original number of features:', features.shape[1])
print('Reduced number of features:', features_kBest.shape[1])

Original number of features: 117
Reduced number of features: 5


In [346]:
# Which five features were selected in step 7? Hint: Use the get_support function.

kbest = np.asarray(mush_x.columns)[chi2_selector.get_support()]

kbest

array(['odor_f', 'odor_n', 'gill-color_b', 'stalk-surface-above-ring_k',
       'stalk-surface-below-ring_k'], dtype=object)

In [349]:
# Repeat steps 4 and 5 with the five best features selected in step 7.
# (Fit a decision tree classifier on the training set.)
# (Report the accuracy and create a confusion matrix for the model prediction on the test set.)

k_x_train, k_x_test, k_y_train, k_y_test = train_test_split(features_kBest, mush_y, test_size = 0.2)

classModel = classifier.fit(k_x_train, k_y_train)

k_y_pred = classModel.predict(k_x_test)

print("Accuracy:",metrics.accuracy_score(k_y_test, k_y_pred))

print(confusion_matrix(k_y_test, k_y_pred))

Accuracy: 0.9218461538461539
[[657 127]
 [  0 841]]


### Summarize your findings.

By using the chi2 selector we limited the features used to the top 5. The accuracy decreased when we limited the features used. However, from the confusion matrix we can see that the decision tree classifier classified all of the poisonous mushrooms correctly. It also incorrectly identified approximately 16% of the edible mushrooms as poisonous. This error is much more acceptable in this scenario than misidentifying poisonous mushrooms as  edible. 

Based on this analysis, it seems using the 5 most impactful features results in a much simpler but still effective decision tree classifier.